In this self study you should experiment with convolutional neural networks using PyTorch. In the last self study session we only made limited use of PyTorch (only using it for calculating gradients), but in this self study we will take advantage of much more of its functionality.

In particular, we will work with the _torch.nn_ module provided by PyTorch. A short introduction to this module and how to define neural networks in PyTorch can be found at

* https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

* https://pytorch.org/tutorials/beginner/nn_tutorial.html

For this self study you may either go through these tutorials before working on the notebook or consult themt when needed as you move forward in the notebook. The former tutorial is part of a general tutorial package to PyTorch, which can be found at (this also includes a nice introduction to tensors in PyTorch)

* https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

First we import relevant modules:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from matplotlib import pyplot
import numpy as np

## Loading the data

As last time we will be working with the MNIST data set: The MNIST database consists of grey scale images of handwritten digits. Each image is of size $28\times 28$; see figure below for an illustration. The data set is divided into a training set consisting of $60000$ images and a test set with $10000$ images; in both
data sets the images are labeled with the correct digits. If interested you can find more information about the MNIST data set at http://yann.lecun.com/exdb/mnist/, including accuracy results for various machine learning methods.

![MNIST DATA](MNIST-dataset.png)

For this self study, we will be a bit more careful with our data. Specifically, we will divide the data into a training, validation, and test, and use the training and validation set for model learning (in the previous self study we did not have a validation set). 

The data set is created by setting aside a randomly chosen subset of the data, where the splitting point is found using the help function *split_indicies* below.

In [2]:
def split_indicies(n, val_pct):
    # Size of validation set
    n_val = int(n*val_pct)
    # Random permutation
    idxs = np.random.permutation(n)
    # Return first indexes for the validation set
    return idxs[n_val:], idxs[:n_val]

# Load the data
train_dataset = datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Resize((28,28)),
                       transforms.RandomRotation(30),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

# Get the indicies for the training data and test data (the validation set will consists of 20% of the data)
train_idxs, val_idxs = split_indicies(len(train_dataset), 0.2)

# Define samplers (used by Dataloader) to the two sets of indicies
train_sampler = SubsetRandomSampler(train_idxs)
val_sampler = SubsetRandomSampler(val_idxs)

# Specify data loaders for our training and test set (same functionality as in the previous self study)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, sampler=val_sampler)

print(f"Number of training examples: {len(train_idxs)}")
print(f"Number of validation examples: {len(val_idxs)}")

Number of training examples: 48000
Number of validation examples: 12000


The test set is loaded in the usual fashion

In [3]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=64, shuffle=True)

## Specifying the model

When using the _torch.nn_ for specifying our model we subclass the _nn.Module_. The model thus holds all the parameters of the model (see the _init_ function) as well as a specification of the forward step. We don't have to keep track of the backward pass, as PyTorch handles this for us.

In [4]:
class MNIST_CNN(nn.Module):

    def __init__(self):
        super().__init__()

        # Define a convolution operator with 1 input channel, 15 output channels and a kernel size of 5x5
        self.conv1 = nn.Conv2d(1, 15, 5)
        # Since we are not doing padding (see Lecture 2, Slide 38) the width of the following layer is reduced; for
        # each channel the resulting dimension is 24x24. We feed the resulting representation through a linear 
        # layer, giving 10 values as output - one for each digit.
        self.fc = nn.Linear(15 * 24 * 24, 10)
        self.out = None

    def forward(self, xb):

        # Reshape the input tensor; '-1' indicates that PyTorch will fill-in this 
        # dimension, whereas the '1' indicates that we only have one color channel. 
        # xb = xb.view(-1, 1, 28, 28) # already done in train loader now
        # Apply convolution and pass the result through a ReLU function
        xb = F.relu(self.conv1(xb))
        # Reshape the representation
        xb = xb.view(-1, 15*24*24)
        # Apply the linear layer
        xb = self.fc(xb)
        # and set the result as the output. Note that we don't take a softmax as this is handled internally in the 
        # loss function defined below.
        self.out = xb

        return xb

## Learning and evaluating the model

For learning the model, we will use the following function which performs one iteration over the training data. The function also takes an _epoch_ argument, but this is only used for reporting on the learning progress

In [5]:
def train(model, train_loader, loss_fn, epoch, optimizer):
    # Tell PyTorch that this function is part of the training
    model.train()

    # As optimizer we use stochastic gradient descent as defined by PyTorch. PyTorch also includes a variety 
    # of other optimizers 
    opt = optimizer#torch.optim.SGD(model.parameters(), lr=learning_rate)

    # Iterate over the training set, one batch at the time, as in the previous self sudy
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get the prediction
        y_pred = model(data)
        
        # Remember to zero the gradients so that they don't accumulate
        opt.zero_grad()

        # Calculate the loss and and the gradients  
        loss = loss_fn(y_pred, target)
        loss.backward()

        # Optimize the parameters by taking one 'step' with the optimizer
        opt.step()

        # For every 10th batch we output a bit of info
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.sampler),
                       100. * batch_idx * len(data) / len(train_loader.sampler), loss.item()))

In the end, we also want to validate our model. To do this we define the function below, which takes a data_loader (either the validation or test set) and reports the model's accuracy and loss on that data set.

In [6]:
def test_model(model, data_loader, loss_fn):
    # Tell PyTorch that we are performing evaluation
    model.eval()
    
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data)
            test_loss += loss_fn(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(data_loader.dataset)

    print('\nTest/validation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.sampler),
        100. * correct / len(data_loader.sampler)))

## A couple of helper functions

Learning a deep neural network can be time consuming, and it might therefore be nice to be able to save and load previously learned models (see also https://pytorch.org/tutorials/beginner/saving_loading_models.html).

In [7]:
def save_model(file_name, model):
    torch.save(model, file_name)

def load_model(file_name):
    model = torch.load(file_name)
    model.eval()
    return model

## Wrapping things up

Finally, we will do the actual learning of the model.

In [8]:
# The number of passes that will be made over the training set
num_epochs = 10
# torch.nn defines several useful loss-functions, which we will take advantage of here (see Lecture 1, Slide 11, Log-loss).
loss_fn = nn.CrossEntropyLoss()

In [9]:
import mnist_architectures
import importlib 
importlib.reload(mnist_architectures)
# Instantiate the model class
# model = MNIST_CNN()
model = mnist_architectures.MNIST_tf()
# model = nn.Sequential(nn.Flatten(start_dim=1), nn.Linear(28*28*1, 10))
# optimizer
lr = 0.001
# optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# and get some information about the structure
print('Model structure:')
print(model)

Model structure:
MNIST_tf(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)


### Iterate over the data set

We iterate over the data set for *num_epochs* number of iterations. At each iteration we also calculate the loss/accuracy on the validation set.

In [10]:
for i in range(num_epochs):
    train(model, train_loader, loss_fn, i, optimizer)
    # Evaluate the model on the test set
    test_model(model, val_loader, loss_fn)

Train Epoch: 0 [0/48000 (0%)]	Loss: 2.306411
Train Epoch: 0 [640/48000 (1%)]	Loss: 1.305019
Train Epoch: 0 [1280/48000 (3%)]	Loss: 0.774701
Train Epoch: 0 [1920/48000 (4%)]	Loss: 0.511080
Train Epoch: 0 [2560/48000 (5%)]	Loss: 0.513153
Train Epoch: 0 [3200/48000 (7%)]	Loss: 0.338646
Train Epoch: 0 [3840/48000 (8%)]	Loss: 0.422407
Train Epoch: 0 [4480/48000 (9%)]	Loss: 0.348527
Train Epoch: 0 [5120/48000 (11%)]	Loss: 0.315359
Train Epoch: 0 [5760/48000 (12%)]	Loss: 0.251678
Train Epoch: 0 [6400/48000 (13%)]	Loss: 0.402013
Train Epoch: 0 [7040/48000 (15%)]	Loss: 0.250289
Train Epoch: 0 [7680/48000 (16%)]	Loss: 0.425981
Train Epoch: 0 [8320/48000 (17%)]	Loss: 0.240673
Train Epoch: 0 [8960/48000 (19%)]	Loss: 0.142264
Train Epoch: 0 [9600/48000 (20%)]	Loss: 0.279521
Train Epoch: 0 [10240/48000 (21%)]	Loss: 0.261882
Train Epoch: 0 [10880/48000 (23%)]	Loss: 0.126158
Train Epoch: 0 [11520/48000 (24%)]	Loss: 0.206934
Train Epoch: 0 [12160/48000 (25%)]	Loss: 0.098449
Train Epoch: 0 [12800/48000 

After learning we evaluate the model on the _test set_ and save the resulting structure.

In [11]:
# Evaluate the model on the test set
test_model(model, test_loader, loss_fn)
# Save the model
save_model(model._get_name()+'.pt', model)


Test/validation set: Average loss: 0.0005, Accuracy: 9909/10000 (99%)



## Exercises

1. Familiarize yourself with the code above and consult the PyTorch documentation when needed.
2. Experiment with different NN architectures (also varying the convolutional parameters: size, stride, padding, etc) and observe the effect wrt. the loss/accuracy on the training and validation dataset (training, validation). Note that when adding new layers (including dropout [Lecture 2, Slide 13], pooling, etc.) you need to make sure that the dimensionality of the layers match up. **IMPORTANT:** ignore the test set at this stage (i.e., comment out the relevant lines above) so that the results for the test set do not influence your model choice.
3. In the model above we use a simple gradient descent learning scheme. Try other types of optimizers (see https://pytorch.org/docs/stable/optim.html) and analyze the effect.
4. Lastly, save your best model and results. At the next lecture we will then see who got the best results :-) Note that for this to be meaningful it is important that you have not relied on the test set while doing model learning/selection.
5. If you feel adventurous, try investigating some of the other datasets that come prepacakged with PyTorch (see https://pytorch.org/vision/0.8/datasets.html). For instnce, for FashionMNIST you only need to change the dataloader from datasets.MNIST to datasets.FashionMNIST.

### Excercise. 2
- Experiment with different NN architectures

__Configuration 1__

Architecture:
  - (conv1): Conv2d(1, 15, kernel_size=(5, 5), stride=(1, 1))
  - Relu
  - (fc): Linear(in_features=8640, out_features=10, bias=True) 

Hyperparameters:
- epochs = 2
- SGD, lr = 0.01

Results:
- Test accuracy: 96%


__Configuration 2__
MNIST_tf

Architecture:
- (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
- Relu, F.max_pool2d(kernel_size=(2, 2))
- (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
- Relu, F.max_pool2d(kernel_size=(2, 2))
- (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
- (fc): Linear(in_features=3136, out_features=10, bias=True)

Hyperparameters:
- epochs = 2
- SGD, lr = 0.01

Results:
- Test accuracy: 96%
- also tried with sigmoid instead of relu, but not as good, only 81%


__Configuration 2__
Linear model

Architecture:
- (0): Flatten(start_dim=1, end_dim=-1)
- (1): Linear(in_features=784, out_features=10, bias=True)


Hyperparameters:
- epochs = 2
- SGD, lr = 0.01

Results:
- Test accuracy: 91%


### Excercise. 3
- We try the Adam optimizer, lr = 0.001, 2 epochs
    - default cnn model it gave 98% accuracy
    - for linear model it gave 92% accuracy
    - for mnist_tf model it gave 99% accuracy

### Excercise. 4
- We try using random affine transformations, with adam lr = 0.001 and the mnist_tf model, 2 epochs
- only 30 degrees random rotation is used
    - default 95% accuracy
    - for linear model it gave 86% accuracy
    - for mnist_tf model it gave 99% accuracy, 10 epochs
    